In [162]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET

In [163]:
buy_out = df(columns=["date",
                        "phone_number",
                        "notebook",
                        "tablet",
                        "phone",
                        "spare_part",
                        "conversation",
                        "customer_price",
                        "our_price",
                        "recycle",
                        "customer_decision"])

map_facts_to_columns = dict()
map_facts_to_columns["phone_number"] = ["CustomerPhone"]
map_facts_to_columns["notebook"] = ["Word", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["phone"] = ["Phone", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["tablet"] = ["Tablet", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["conversation"] = ["Buy"]
map_facts_to_columns["recycle"] = ["source sentence"]
map_facts_to_columns["customer_decision"] = ["what_customer_expects"]

buy_out

,date,phone_number,notebook,tablet,phone,spare_part,conversation,customer_price,our_price,recycle,customer_decision


In [164]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s");
        text = "";
        for i in source_sentence.itertext():
            text += i;
        dict_of_leads[id] = text;
    return dict_of_leads;

make_dict_of_leads()["46"]

'89140580517 продажа айфон6s на выезде Ляпидевского 12.'

In [167]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

calls = df(columns=[
        "CustomerPhone_Phone",
        "FromUU",
        "Buy_Word",
        "Pawn_Word",
        "Notebook_Word",
        "Notebook_Vendor",
        "Notebook_Model",
        "Notebook_UserDefinedName",
        "Phone_Word",
        "Phone_Vendor",
        "Phone_Model",
        "Phone_UserDefinedName",
        "source_sentence"
    ])

facts = compare_facts_to_leads()
leads = make_dict_of_leads()
for lead in facts:
    elems = facts[lead]
    one_sentence = lead
    columns = []
    values = []
    for fact_name in elems:
        for fact_field in fact_name:
            columns.append(fact_name.tag + "_" + fact_field.tag)
            values.append(fact_field.attrib["val"])
    print("_____________")            
    print(lead, values, columns)
    one_row = pd.DataFrame(values, columns=columns)
    print("_____________")
    values = []
    columns = []
#         
#         columns = []
#         values = []
#     print(one_row)
#             one_sentence += " | >" + i.attrib["val"] + "<"
#         one_sentence += " | " + e.tag
#         if e.tag.lower() == "BUY".lower():
#             print(i.tag, one_sentence)
#             print(lead, leads[lead])
    

            

_____________
46 ['89140580517', 'АЙФОН6S'] ['CustomerPhone_Phone', 'Phone_UserDefinedName']
_____________
_____________
24 ['НОУТБУК'] ['Notebook_Word']
_____________
_____________
53 ['89140580517', 'ACER'] ['CustomerPhone_Phone', 'Notebook_Vendor']
_____________
_____________
18 ['PAWN'] ['Pawn_Word']
_____________
_____________
10 ['BUY', '17', '89140580517'] ['Buy_Word', 'Buy_NumberInPrice', 'CustomerPhone_Phone']
_____________
_____________
28 ['89140580517', 'АСЕР'] ['CustomerPhone_Phone', 'Notebook_Vendor']
_____________
_____________
54 ['89140580517'] ['CustomerPhone_Phone']
_____________
_____________
30 ['89140580517', 'АСЕР'] ['CustomerPhone_Phone', 'Notebook_Vendor']
_____________
_____________
3 ['89140580517', 'УУ', 'НОУТБУК', 'BUY'] ['CustomerPhone_Phone', 'CustomerPlace_Place', 'Notebook_Word', 'Buy_Word']
_____________
_____________
19 ['НОУТБУК', 'АСЕР', '89140580517', 'PAWN'] ['Notebook_Word', 'Notebook_Vendor', 'CustomerPhone_Phone', 'Pawn_Word']
_____________
___